In [54]:
import pandas as pd
from transformers import AutoModel

In [55]:
train_raw = pd.read_json('data/en_ewt-up-train.jsonl',lines=True)
dev_raw = pd.read_json('data/en_ewt-up-dev.jsonl',lines=True)
test_raw = pd.read_json('data/en_ewt-up-test.jsonl',lines=True)

In [56]:
print(train_raw)

                                                                                                                                                                    seq_words   
0      [Al, -, Zaman, :, American, forces, killed, Shaikh, Abdullah, al, -, Ani, ,, the, preacher, at, the, mosque, in, the, town, of, Qaim, ,, near, the, Syrian, border, .]  \
1                                                                    [[, This, killing, of, a, respected, cleric, will, be, causing, us, trouble, for, years, to, come, ., ]]   
2                                                                    [[, This, killing, of, a, respected, cleric, will, be, causing, us, trouble, for, years, to, come, ., ]]   
3                                                                    [[, This, killing, of, a, respected, cleric, will, be, causing, us, trouble, for, years, to, come, ., ]]   
4                                                                    [[, This, killing, of, a, respected, cleric, w

The silly token_classification preproc code requires a space separated list of ints, with one per token including punctuation.

In [57]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
print(train_raw['BIO'].head(500).tail(50))

450                                                                                                                                                                      [O]
451                                                                                                                                                                      [O]
452                                                                                                                                                                      [O]
453                                                                                                                                                                      [O]
454                                                                                                                                                                      [O]
455                                                                                                                                    

In [58]:
letcount = {}
for y in train_raw['BIO']:
    for xx in y:
        x = xx.replace("-C-","-").replace("-R-","-")
        if x not in letcount:
            letcount[x] = 1
        else:
            letcount[x] +=1
qq = (sorted(letcount.items(), key=lambda q: q[1]))
print(qq)

[('B-ARGA', 1), ('B-ARG5', 4), ('B-ARGM-REC', 24), ('B-ARG1-DSP', 47), ('B-ARGM-COM', 114), ('B-ARGM-CXN', 200), ('B-ARGM-GOL', 227), ('B-ARG4', 414), ('B-ARGM-PRD', 459), ('B-ARGM-DIR', 500), ('B-ARGM-LVB', 553), ('B-ARGM-PRR', 556), ('B-ARGM-CAU', 607), ('B-ARG3', 682), ('B-ARGM-PRP', 721), ('B-ARGM-EXT', 875), ('B-ARGM-DIS', 1426), ('B-ARGM-MNR', 1501), ('B-ARGM-ADJ', 1691), ('B-ARGM-LOC', 1697), ('B-ARGM-NEG', 1909), ('B-ARGM-MOD', 3570), ('B-ARGM-ADV', 4462), ('B-ARGM-TMP', 5374), ('B-ARG2', 9959), ('B-ARG0', 15953), ('B-ARG1', 28772), ('B-V', 40661), ('O', 912945)]


In [59]:
label_map = {'B-ARG3': 3, 'B-ARGM-MOD': 8, 'B-ARGM-ADV': 7, 'B-ARGM-TMP': 6, 'B-ARG2': 2, 'B-ARG0': 0, 'B-ARG1':1, 'B-V': 4, 'O':5}

label_map_rev = {v: k for k,v in label_map.items()}

def label2id(label):
    label = label.replace("-C-","-").replace("-R-","-")
    return label_map.get(label, 5)

def check_lens(row):
    assert len(row['seq_words']) == len(row['labels'])
    return True


print(label_map, label_map_rev)
train_raw['labels'] = train_raw['BIO'].apply(lambda q: list(map(label2id, q)))
train_raw['checked'] = train_raw.apply(check_lens, axis=1)

{'B-ARG3': 3, 'B-ARGM-MOD': 8, 'B-ARGM-ADV': 7, 'B-ARGM-TMP': 6, 'B-ARG2': 2, 'B-ARG0': 0, 'B-ARG1': 1, 'B-V': 4, 'O': 5} {3: 'B-ARG3', 8: 'B-ARGM-MOD', 7: 'B-ARGM-ADV', 6: 'B-ARGM-TMP', 2: 'B-ARG2', 0: 'B-ARG0', 1: 'B-ARG1', 4: 'B-V', 5: 'O'}


In [63]:
from datasets import load_dataset

wnut = load_dataset("wnut_17")
example = wnut["train"][0]
print(example)


Found cached dataset wnut_17 (/home/titan-0/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9)


  0%|          | 0/3 [00:00<?, ?it/s]

{'id': '0', 'tokens': ['@paulwalk', 'It', "'s", 'the', 'view', 'from', 'where', 'I', "'m", 'living', 'for', 'two', 'weeks', '.', 'Empire', 'State', 'Building', '=', 'ESB', '.', 'Pretty', 'bad', 'storm', 'here', 'last', 'evening', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]}


In [81]:
from transformers import AutoTokenizer, BertForSequenceClassification
import torch


#each row is a dict, with a tokens/seq_words entry, which is a list of words, and a labels entry, which is a list of ints.

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))
model = model.to('cuda:0')
# Preprocess data and tokenize input texts

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    print(labels)
    print(torch.tensor(tokenized_inputs['input_ids']))

    tokenized_inputs["labels"] = torch.tensor(labels)
    return tokenized_inputs


data = {}
data['tokens'] = train_raw[:30]['seq_words'].tolist()
data['labels'] = train_raw[:30]['labels'].tolist()

inputs = tokenize_and_align_labels(data)

print(inputs)

from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
data
# Training loop
num_epochs = 3
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)


for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(data_collator)
    loss = outputs.loss
    logits = outputs.logits
    loss.backward()
    optimizer.step()








# Save the trained model
#model.save_pretrained('hf_trained_srl')



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

[[-100, 5, 5, 5, -100, 5, 5, 0, 4, 1, -100, 5, 5, 5, 5, -100, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, -100, -100, 5, 5, 5, 5, 5, 5, -100], [-100, 5, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, -100], [-100, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, -100], [-100, 5, 5, 0, 5, 5, 5, 5, 8, 5, 4, 5, 1, 5, 6, 5, 5, 5, 5, -100], [-100, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 4, 5, 5, -100], [-100, 5, -100, 5, 5, 0, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, -100], [-100, 5, -100, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, -100], [-100, 5, -100, 5, 5, 5, 5, 5, 0, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, -100], [-100, 5, -100, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 4, 5, 5, 5, -100], [-100, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, -100], [-100, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, -100], [-100, 1, 5, 5, 5, 5, 4, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, -100], [-100, 5, 1, 5, 5, 4, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, -100, 5, 5, 5, -100, -100, -100, 5, 5, 5, 5, 5, 5, 5, 5, 5, -100,

ValueError: expected sequence of length 36 at dim 1 (got 20)